In [2]:
# Install necessary libraries
!pip install geopy haversine ortools pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [3]:
 # Import required libraries
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from haversine import haversine
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp



In [4]:

# Read different sheets
vehicle_data = pd.read_excel("SmartRoute Optimizer.xlsx", sheet_name="Vehicle_Information")
shipments_data = pd.read_excel("SmartRoute Optimizer.xlsx", sheet_name="Shipments_Data")
store_location = pd.read_excel("SmartRoute Optimizer.xlsx", sheet_name="Store Location")

# Display first few rows
print("Vehicle Data:")
display(vehicle_data.head())

print("Shipments Data:")
display(shipments_data.head())

print("Store Location:")
display(store_location.head())

Vehicle Data:


,Vehicle Type,Number,Shipments_Capacity,Max Trip Radius (in KM)
0,3W,50,5,15
1,4W-EV,25,8,20
2,4W,Any,25,Any


Shipments Data:


,Shipment ID,Latitude,Longitude,Delivery Timeslot,weight
0,1,19.080268,72.850804,09:30:00-12:00:00,10
1,2,19.112110,72.898355,09:30:00-12:00:00,10
2,3,19.058145,72.834377,07:00:00-09:30:00,10
3,4,19.176505,72.962189,12:00:00-14:30:00,10
4,5,19.148967,72.931665,09:30:00-12:00:00,10


Store Location:


,Latitute,Longitude
0,19.075887,72.877911


In [5]:
# Check the first few rows of the 'Delivery Timeslot' column to understand its format
print(shipments_data['Delivery Timeslot'].head())

0    09:30:00-12:00:00
1    09:30:00-12:00:00
2    07:00:00-09:30:00
3    12:00:00-14:30:00
4    09:30:00-12:00:00
Name: Delivery Timeslot, dtype: object


In [6]:
# Convert delivery timeslot to a more usable format
def parse_time_slot(timeslot):
    try:
        # Strip any leading/trailing whitespace and split by '-'
        start_time_str, end_time_str = timeslot.strip().split('-')

        # Remove ':00' if present and return the hour part as an integer
        start_hour = int(start_time_str.replace(':00', '').replace(':30', ''))
        end_hour = int(end_time_str.replace(':00', '').replace(':30', ''))

        return start_hour, end_hour
    except Exception as e:
        print(f"Error processing timeslot {timeslot}: {e}")
        return None, None  # Handle cases where the parsing fails

# Apply the new parsing function to the 'Delivery Timeslot' column
shipments_data[['Start_Time', 'End_Time']] = shipments_data['Delivery Timeslot'].apply(lambda x: pd.Series(parse_time_slot(x)))

# Check the first few rows of the 'Delivery Timeslot' column to understand its format
print(shipments_data['Delivery Timeslot'].head())

# Convert delivery timeslot to a more usable format
def parse_time_slot(timeslot):
    try:
        # Strip any leading/trailing whitespace and split by '-'
        start_time_str, end_time_str = timeslot.strip().split('-')

        # Remove ':00' if present and return the hour part as an integer
        start_hour = int(start_time_str.replace(':00', '').replace(':30', ''))
        end_hour = int(end_time_str.replace(':00', '').replace(':30', ''))

        return start_hour, end_hour
    except Exception as e:
        print(f"Error processing timeslot {timeslot}: {e}")
        return None, None  # Handle cases where the parsing fails

# Apply the new parsing function to the 'Delivery Timeslot' column
shipments_data[['Start_Time', 'End_Time']] = shipments_data['Delivery Timeslot'].apply(lambda x: pd.Series(parse_time_slot(x)))

vehicle_data['Shipments_Capacity'] = vehicle_data['Shipments_Capacity'].astype(float)

# Store Latitude and Longitude
store_lat, store_long = store_location.iloc[0]['Latitute'], store_location.iloc[0]['Longitude']

# Display the cleaned data
print("Preprocessed Shipments Data:")
display(shipments_data.head())

print("Preprocessed Vehicle Data:")
display(vehicle_data.head())


0    09:30:00-12:00:00
1    09:30:00-12:00:00
2    07:00:00-09:30:00
3    12:00:00-14:30:00
4    09:30:00-12:00:00
Name: Delivery Timeslot, dtype: object
Preprocessed Shipments Data:


,Shipment ID,Latitude,Longitude,Delivery Timeslot,weight,Start_Time,End_Time
0,1,19.080268,72.850804,09:30:00-12:00:00,10,9,12
1,2,19.112110,72.898355,09:30:00-12:00:00,10,9,12
2,3,19.058145,72.834377,07:00:00-09:30:00,10,7,9
3,4,19.176505,72.962189,12:00:00-14:30:00,10,12,14
4,5,19.148967,72.931665,09:30:00-12:00:00,10,9,12


Preprocessed Vehicle Data:


,Vehicle Type,Number,Shipments_Capacity,Max Trip Radius (in KM)
0,3W,50,5.0,15
1,4W-EV,25,8.0,20
2,4W,Any,25.0,Any


In [7]:
# Function to calculate distance using haversine formula
def calculate_distance_matrix(locations):
    size = len(locations)
    distance_matrix = np.zeros((size, size))

    for i in range(size):
        for j in range(size):
            distance_matrix[i][j] = haversine(locations[i], locations[j])

    return distance_matrix

# Combine store and shipment locations
locations = [(store_lat, store_long)] + list(zip(shipments_data['Latitude'], shipments_data['Longitude']))
distance_matrix = calculate_distance_matrix(locations)

# Display distance matrix
print("Distance Matrix:")
print(distance_matrix)

Distance Matrix:
[[ 0.          2.88995529  4.56489726 ...  7.97690446  2.66587116
  10.08513969]
 [ 2.88995529  0.          6.12383242 ...  7.30773537  0.22454995
   9.64304988]
 [ 4.56489726  6.12383242  0.         ... 12.52889095  5.95747405
  14.6491913 ]
 ...
 [ 7.97690446  7.30773537 12.52889095 ...  0.          7.33496051
   2.35676979]
 [ 2.66587116  0.22454995  5.95747405 ...  7.33496051  0.
   9.66101869]
 [10.08513969  9.64304988 14.6491913  ...  2.35676979  9.66101869
   0.        ]]


In [8]:
# Define vehicle capacity constraints
vehicle_capacities = list(vehicle_data['Shipments_Capacity'])

# Define vehicle distance limits (Fixing the column name)
vehicle_distance_limits = list(vehicle_data['Max Trip Radius (in KM)'])

# Define number of vehicles available
num_vehicles = len(vehicle_data)

# Display vehicle details
print("Vehicle Capacities:", vehicle_capacities)
print("Vehicle Distance Limits:", vehicle_distance_limits)
print("Total Vehicles:", num_vehicles)

Vehicle Capacities: [5.0, 8.0, 25.0]
Vehicle Distance Limits: [15, 20, 'Any']
Total Vehicles: 3


In [9]:

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix.tolist()  # Distance matrix
    data['num_vehicles'] = num_vehicles  # Number of vehicles
    data['vehicle_capacities'] = vehicle_capacities  # Vehicle capacities
    data['vehicle_distance_limits'] = vehicle_distance_limits  # Vehicle max distance
    data['demands'] = [0] + list(shipments_data['weight'])  # Shipment weights
    data['time_windows'] = [(0, 24)] + list(zip(shipments_data['Start_Time'], shipments_data['End_Time']))  # Time slots
    data['depot'] = 0  # Store as the starting point
    return data

data = create_data_model()

In [10]:
# Ensure vehicle capacities are integers
data['vehicle_capacities'] = [int(capacity) for capacity in data['vehicle_capacities']]

In [19]:
def solve_vrp(data):
    """Solves the vehicle routing problem with a time limit of 4 minutes."""
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
      """Returns the distance between nodes safely."""
      try:
          from_node = manager.IndexToNode(int(from_index))  # Ensure index is int
          to_node = manager.IndexToNode(int(to_index))      # Ensure index is int

          # Debugging Prints
          print(f"from_index: {from_index}, to_index: {to_index}")
          print(f"from_node: {from_node}, to_node: {to_node}")

          return int(data['distance_matrix'][from_node][to_node])  # Convert float to int
      except (OverflowError, TypeError) as e:
          print(f"Error occurred: {e}")  # Print exact error
          return 0  # Prevent crashes

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint
    def demand_callback(from_index):
        """Returns the demand of the node."""
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, data['vehicle_capacities'], True, "Capacity")

    def time_callback(from_index, to_index):
      """Returns the travel time between nodes safely."""
      try:
         return int(distance_callback(from_index, to_index) // 50)  # Ensure integer division
      except (OverflowError, TypeError) as e:
          print(f"Error in time_callback: {e}")
          return 0



    time_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.AddDimension(time_callback_index, 0, 24, True, "Time")

    # Define search parameters with time limit
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    # Set a time limit of 4 minutes (240 seconds)
    search_parameters.time_limit.seconds = 240

    # Set a solution limit to stop excessive computation
    search_parameters.solution_limit = 100000

    # Set local search time limit
    search_parameters.lns_time_limit.seconds = 10

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print("Solution Found!")
        return solution, manager, routing
    else:
        print("No Solution Found within 4 minutes.")
        return None, None, None


In [20]:
solution, manager, routing = solve_vrp(data)

Streaming output truncated to the last 5000 lines.
from_node: 0, to_node: 847
from_index: 1220, to_index: 849
from_node: 0, to_node: 849
from_index: 1220, to_index: 850
from_node: 0, to_node: 850
from_index: 1220, to_index: 853
from_node: 0, to_node: 853
from_index: 1220, to_index: 854
from_node: 0, to_node: 854
from_index: 1220, to_index: 855
from_node: 0, to_node: 855
from_index: 1220, to_index: 857
from_node: 0, to_node: 857
from_index: 1220, to_index: 859
from_node: 0, to_node: 859
from_index: 1220, to_index: 860
from_node: 0, to_node: 860
from_index: 1220, to_index: 861
from_node: 0, to_node: 861
from_index: 1220, to_index: 862
from_node: 0, to_node: 862
from_index: 1220, to_index: 863
from_node: 0, to_node: 863
from_index: 1220, to_index: 865
from_node: 0, to_node: 865
from_index: 1220, to_index: 866
from_node: 0, to_node: 866
from_index: 1220, to_index: 867
from_node: 0, to_node: 867
from_index: 1220, to_index: 868
from_node: 0, to_node: 868
from_index: 1220, to_index: 869
from_

In [21]:
def print_solution(manager, routing, solution):
    """Prints the solution."""
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        route = []

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        print(f"Route for Vehicle {vehicle_id}: {route}")
        print(f"Distance of the route: {route_distance/1000} km")

if solution:
    print_solution(manager, routing, solution)